In [1]:
import pandas as pd
def descomprimir(archivo):
    df = pd.read_csv(archivo,index_col=False, compression='zip')
    return df

data_test = descomprimir('../files/input/test_data.csv.zip')
data_train = descomprimir('../files/input/train_data.csv.zip')

#### Paso 1.
##### Preprocese los datos.
- Cree la columna 'Age' a partir de la columna 'Year'.
- Asuma que el año actual es 2021.
- Elimine las columnas 'Year' y 'Car_Name'.

In [2]:
def preprocesamiento(data):
    data['Age'] = data['Year'].map(lambda x : 2021 - x)
    data = data.drop(columns = ['Year','Car_Name'])
    return data
data_test = preprocesamiento(data_test)
data_train = preprocesamiento(data_train)

#### Paso 2.
##### Divida los datasets en x_train, y_train, x_test, y_test.

In [3]:
x_train = data_train.drop(columns = ['Present_Price'])
y_train = data_train['Present_Price']
x_test = data_test.drop(columns = ['Present_Price'])
y_test = data_test['Present_Price']

In [4]:
print(x_train.head())
print(y_train.head())

   Selling_Price  Driven_kms Fuel_Type Selling_type Transmission  Owner  Age
0           7.40       15059    Petrol       Dealer    Automatic      0    5
1           4.00       30000    Petrol       Dealer       Manual      0    8
2           0.50        6000    Petrol   Individual       Manual      0   10
3           3.15       15000    Petrol       Dealer       Manual      0    5
4           1.25       15000    Petrol   Individual       Manual      0    8
0    8.500
1    4.600
2    0.826
3    4.430
4    1.500
Name: Present_Price, dtype: float64


#### Paso 3.
##### Cree un pipeline para el modelo de clasificación. Este pipeline debe contener las siguientes capas:
- Transforma las variables categoricas usando el método one-hot-encoding.
- Escala las variables numéricas al intervalo [0, 1].
- Selecciona las K mejores entradas.
- Ajusta un modelo de regresion lineal.

In [19]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression

categorical_features = ['Fuel_Type','Selling_type','Transmission']


preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features),('num', MinMaxScaler(), ['Selling_Price','Driven_kms','Owner','Age'])
    ],
    remainder='passthrough'
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('Skbest', SelectKBest(score_func=f_regression)),
    ('model', LinearRegression())
])

#### Paso 4.
- Optimice los hiperparametros del pipeline usando validación cruzada.
- Use 10 splits para la validación cruzada. Use el error medio absoluto para medir el desempeño modelo.

In [20]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'Skbest__k': range(1, 20, 2),
    'model__fit_intercept': [True, False],
    'model__copy_X': [True, False],
    'model__n_jobs': [-1],
    'model__positive': [True, False],

}

grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error',
    cv=10,
    n_jobs=-1
)

grid_search.fit(x_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('cat',
                                                                         OneHotEncoder(),
                                                                         ['Fuel_Type',
                                                                          'Selling_type',
                                                                          'Transmission']),
                                                                        ('num',
                                                                         MinMaxScaler(),
                                                                         ['Selling_Price',
                                                                          'Driven_kms',
                                                                          'Owner',
                                                                          'Age'])])),
                                       ('Skbest',
                                        SelectKBest(score_func=<function f_regression at 0x0000027B4A1DB600>)),
                                       ('model', LinearRegression())]),
             n_jobs=-1,
             param_grid={'Skbest__k': range(1, 20, 2),
                         'model__copy_X': [True, False],
                         'model__fit_intercept': [True, False],
                         'model__n_jobs': [-1],
                         'model__positive': [True, False]},
             scoring='neg_mean_absolute_error')

In [21]:
##paso 5
import gzip
import pickle
import os
# Crear carpeta si no existe
output_dir = '../files/models'
os.makedirs(output_dir, exist_ok=True)

compressed_model_path = '../files/models/model.pkl.gz'

with gzip.open(compressed_model_path, 'wb') as f:
    pickle.dump(grid_search, f)

#### Paso 6.
##### Calcule las metricas r2, error cuadratico medio, y error absoluto medio para los conjuntos de entrenamiento y prueba. 
##### Guardelas en el archivo files/output/metrics.json. Cada fila del archivo es un diccionario con las metricas de un modelo. ##### Este diccionario tiene un campo para indicar si es el conjunto de entrenamiento o prueba. Por ejemplo:

{'type': 'metrics', 'dataset': 'train', 'r2': 0.8, 'mse': 0.7, 'mad': 0.9}
{'type': 'metrics', 'dataset': 'test', 'r2': 0.7, 'mse': 0.6, 'mad': 0.8}


In [22]:
## paso 6
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import json

y_train_pred = grid_search.predict(x_train) 
y_test_pred = grid_search.predict(x_test)

# Calcular las métricas para el conjunto de entrenamiento, en la primera va el nombre del dataset y en la segunda el train 
train_metrics = { 'type': "metrics",
    'dataset': 'train', 
                'r2': float(r2_score(y_train, y_train_pred)),
                'mse': float(mean_squared_error(y_train, y_train_pred)), 
                'mad': float(mean_absolute_error(y_train, y_train_pred))} 

# Calcular las métricas para el conjunto de prueba 
test_metrics = { 'type': "metrics",
    'dataset': 'test', 
                'r2': float(r2_score(y_test, y_test_pred)), 
                'mse': float(mean_squared_error(y_test, y_test_pred)),
                'mad': float(mean_absolute_error(y_test, y_test_pred)) }

print("Metricas de entrenamiento")
print(json.dumps(train_metrics, indent=4))
print("Metricas de prueba")
print(json.dumps(test_metrics, indent=4))

Metricas de entrenamiento
{
    "type": "metrics",
    "dataset": "train",
    "r2": 0.8916962358587399,
    "mse": 5.8746462805982045,
    "mad": 1.6214879816158285
}
Metricas de prueba
{
    "type": "metrics",
    "dataset": "test",
    "r2": 0.7325716754123306,
    "mse": 32.56667275386626,
    "mad": 2.4736199144163757
}


In [23]:
# Crear carpeta si no existe
output_dir = '../files/output'
os.makedirs(output_dir, exist_ok=True)

# Guardar las métricas en un archivo JSON
output_path = '../files/output/metrics.json'
with open(output_path, 'w') as f:  # Usar 'w' para comenzar con un archivo limpio
        f.write(json.dumps(train_metrics) + '\n')
        f.write(json.dumps(test_metrics) + '\n')

print(f"Métricas guardadas en {output_path}")

Métricas guardadas en ../files/output/metrics.json
